In [1]:
import torch
from nnsight import LanguageModel
from utils import utils

In [2]:
device = 'cuda:0'

# set random seeds
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [3]:
# model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
# hidden_dim = 4096

model_name = 'google/gemma-2b-it'
hidden_dim = 2048

model = LanguageModel(model_name, device_map=device)

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


In [4]:
model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR

In [5]:
base_prompt = "Complete the following review. 'This movie was awesome, I loved it. The movie's genre of"
weight = 0.
direction = torch.randn(hidden_dim).to(device)
max_tokens = 3

utils.generate_nnsight(model, device, base_prompt, weight, direction, max_tokens)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


"<bos>Complete the following review. 'This movie was awesome, I loved it. The movie's genre of science fiction,"

In [6]:
base_prompt = "Complete the following review. 'This movie was awesome, I loved it. The movie's genre of"
weight = 10.
direction = torch.randn(hidden_dim).to(device)
max_tokens = 3

utils.generate_nnsight(model, device, base_prompt, weight, direction, max_tokens)

"<bos>Complete the following review. 'This movie was awesome, I loved it. The movie's genre of embodi embodi embodi"

### Load some actual directions

In [7]:
saved_directions = f"/net/projects/veitch/geometry_llms/directions/intervention/sentiment_{model_name.split('/')[-1]}.pt"
# load the directions
directions = torch.load(saved_directions)
print(directions.shape)

torch.Size([5, 2048])


In [8]:
weight = 50.
for direction in directions:
    generation = utils.generate_nnsight(model, device, base_prompt, weight, direction, max_tokens)
    print(generation)

<bos>Complete the following review. 'This movie was awesome, I loved it. The movie's genre of magical realism seamlessly
<bos>Complete the following review. 'This movie was awesome, I loved it. The movie's genre of sci-fi
<bos>Complete the following review. 'This movie was awesome, I loved it. The movie's genre of sci-fi
<bos>Complete the following review. 'This movie was awesome, I loved it. The movie's genre of<bos><bos><bos>
<bos>Complete the following review. 'This movie was awesome, I loved it. The movie's genre of great great great


haha I wonder which is which...